In [ ]:
# import libraries
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
from tqdm import tqdm

In [4]:
class BacteriaGrowthModel():
  def __init__(self,
               size: int,
               b_density: float,
               f_density: float,
               initial_bacteria: float,
               initial_food: float,
               p_add_food: float,
               food_diff_rate: float,
               bacteria_diff_rate: float,
               bacteria_growth_rate: float,
               food_growth_rate: float,
               consumption_rate: float,
               max_food_capacity: float):

    # initial parameters
    self.size = size
    self.current_state = np.zeros((size, size, 2)) # square matrix where each cell contains an array of two numbers
    self.next_state = np.zeros((size, size, 2)) # square matrix where each cell contains an array of two numbers

    self.b_density = b_density # proportion of cells to add bacteria at first
    self.f_density = f_density # proportion of cells to add food at first
    self.initial_food = initial_food
    self.initial_bacteria = initial_bacteria

    # constraint
    self.max_food_capacity = 100
    self.p_add_food = p_add_food

    # diffusion rates
    self.food_diff_rate = food_diff_rate
    self.bacteria_diff_rate = bacteria_diff_rate

    # growth rates
    self.bacteria_growth_rate = bacteria_growth_rate
    self.food_growth_rate = food_growth_rate

    # consumption rate
    self.consumption_rate = consumption_rate

    # step counter for visualization
    self.step_counter = 0

  def initialize(self):
    """
    Scatter food and bacteria randomly
    """
    num_cells = self.size ** 2
    bacteria_indices = np.random.choice(a=range(num_cells),
                                        size=int(round(self.b_density*num_cells, 1) + 1),
                                        replace=False)
    food_indices = np.random.choice(a=range(num_cells),
                                    size=int(round(self.f_density*num_cells, 1) + 1),
                                    replace=False)

    current_state_1d = self.current_state[:, :, 0].reshape(num_cells)
    for i in bacteria_indices:
      current_state_1d[i] = self.initial_bacteria
    self.current_state[:, :, 0] = current_state_1d.reshape(self.size, self.size)

    current_state_1d = self.current_state[:, :, 1].reshape(num_cells)
    for j in food_indices:
      current_state_1d[j] = self.initial_food
    self.current_state[:, :, 1] = current_state_1d.reshape(self.size, self.size)

    self.figure, self.axes = plt.subplots()

  def draw(self):
    plot = self.axes.imshow(
        self.current_state[:, :, 0], vmin=0, vmax=1, cmap='magma') # change to 0 to visualize bacteria
    self.axes.set_title(f'State at step {self.step_counter}')
    return plot

  def grow_food(self):
    self.next_state = np.copy(self.current_state)

    food_indices = np.where(self.current_state[:, :, 1] < self.max_food_capacity) # get an array of indices where the food is less than the constraint
    food_values = self.current_state[food_indices[0], food_indices[1], 1]
    growth_factor = 1 + self.food_growth_rate * (1 - food_values / self.max_food_capacity)

    # update the current state
    self.next_state[food_indices[0], food_indices[1], 1] = food_values * growth_factor
    self.current_state, self.next_state = self.next_state, self.current_state
    self.step_counter += 1

  def reseed_food(self):
    self.next_state = np.copy(self.current_state)

    # randomly choose which indices to add food
    mask = np.random.uniform(0, 1, (self.size, self.size)) < self.p_add_food

    # update the state
    self.next_state[:, :, 1][mask] += 1
    self.current_state, self.next_state = self.next_state, self.current_state
    self.step_counter += 1

  def diffuse_food(self):
    self.next_state = np.copy(self.current_state)

    for row in range(self.size):
      for col in range(self.size):
        removed_food = self.food_diff_rate * self.current_state[row, col, 1]
        self.next_state[row, col, 1] -= removed_food

        for dx, dy in [(0,1), (0,-1), (1,0), (-1,0)]:
          self.next_state[(row + dx) % self.size, (col + dy) % self.size, 1] += (1/4) * removed_food

    self.current_state, self.next_state = self.next_state, self.current_state
    self.step_counter += 1

  def consume_food(self):
    self.next_state = np.copy(self.current_state)

    for row in range(self.size):
      for col in range(self.size):
        required_food = self.consumption_rate * self.current_state[row, col, 0]
        food_consumed = np.minimum(self.current_state[row, col, 1], required_food)
        bacteria_survived = food_consumed / self.consumption_rate

        self.next_state[row, col, 0] = bacteria_survived
        self.next_state[row, col, 1] -= food_consumed

    self.current_state, self.next_state = self.next_state, self.current_state
    self.step_counter += 1

  def starvation(self):
    self.step_counter += 1

  def reproduce_bacteria(self):
    self.next_state = np.copy(self.current_state)

    for row in range(self.size):
      for col in range(self.size):
        offspring = self.bacteria_growth_rate * self.current_state[row, col, 0]
        self.next_state[row, col, 0] = self.current_state[row, col, 0] * (1 + self.bacteria_growth_rate)

    self.current_state, self.next_state = self.next_state, self.current_state
    self.step_counter += 1

  def diffuse_bacteria(self):
    self.next_state = np.copy(self.current_state)

    for row in range(self.size):
      for col in range(self.size):
        removed_bacteria = self.bacteria_diff_rate * self.current_state[row, col, 0]
        self.next_state[row, col, 0] -= removed_bacteria

        for dx, dy in [(0,1), (0,-1), (1,0), (-1,0)]:
          self.next_state[(row + dx) % self.size, (col + dy) % self.size, 0] += (1/4) * removed_bacteria

    self.current_state, self.next_state = self.next_state, self.current_state
    self.step_counter += 1

def update(frame, sim, steps_per_frame, progress_bar):
    for _ in range(steps_per_frame):
        sim.grow_food()
        sim.reseed_food()
        sim.diffuse_food()
        sim.consume_food()
        sim.starvation()
        sim.reproduce_bacteria()
        sim.diffuse_bacteria()
    plot = sim.draw()
    progress_bar.update(1)
    return [plot]

def make_animation(sim, total_frames, steps_per_frame, interval=100):
    frame_number = 0
    sim.initialize()
    progress_bar = tqdm(total=total_frames)
    update(frame_number, sim, steps_per_frame, progress_bar)
    animation = FuncAnimation(
        sim.figure,
        update,
        fargs=(sim, steps_per_frame, progress_bar),
        init_func=lambda: [],
        frames=total_frames,
        interval=interval,
        blit=True
    )
    output = HTML(animation.to_html5_video())
    sim.figure.clf()
    plt.close(sim.figure)
    return output

In [5]:
def run_simulation():
    """Run a bacteria growth simulation with specified parameters"""
    model = BacteriaGrowthModel(
                        size=100,                  # Grid size matching your image (200x200)
                        b_density=0.5,            # Lower initial bacteria density
                        f_density=0.5,             # Moderate food density
                        initial_bacteria=0.5,      # Higher initial bacteria amount where placed
                        initial_food=0.5,          # Higher initial food
                        p_add_food=0.01,           # Standard reseeding probability
                        food_diff_rate=0.5,        # Very high food diffusion
                        bacteria_diff_rate=0.5,    # Low bacteria diffusion (critical ratio)
                        bacteria_growth_rate=0.5,  # Moderate bacteria growth
                        food_growth_rate=0.5,     # Moderate food growth
                        consumption_rate=0.5,      # Lower consumption rate
                        max_food_capacity=100      # Standard capacity
)
    return make_animation(model, total_frames=100, steps_per_frame=5)

# Run for enough time to develop patterns
animation = run_simulation()
animation

101it [00:45,  1.51it/s]                         